In [41]:
from keras.preprocessing import text
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import nltk
from nltk.corpus import gutenberg
from string import punctuation
import re

In [2]:
#texts = pd.read_csv("북촌계획110-120.csv",engine="python")

In [38]:
#nltk.download("gutenberg")
#ltk.download("punkt")
nltk.download("stopwords")
#bible = gutenberg.sents("bible-kjv.txt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [39]:

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [47]:
bible = gutenberg.sents('bible-kjv.txt') 
remove_terms = punctuation + '0123456789'

norm_bible = [[word.lower() for word in sent if word not in remove_terms] for sent in bible]
norm_bible = [' '.join(tok_sent) for tok_sent in norm_bible]
norm_bible = filter(None, normalize_corpus(norm_bible))
norm_bible = [tok_sent for tok_sent in norm_bible if len(tok_sent.split()) > 2]

print('Total lines:', len(bible))
print('\nSample line:', bible[10])
print('\nProcessed line:', norm_bible[10])

Total lines: 30103

Sample line: ['1', ':', '6', 'And', 'God', 'said', ',', 'Let', 'there', 'be', 'a', 'firmament', 'in', 'the', 'midst', 'of', 'the', 'waters', ',', 'and', 'let', 'it', 'divide', 'the', 'waters', 'from', 'the', 'waters', '.']

Processed line: god said let firmament midst waters let divide waters waters


In [48]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)

In [49]:
word2id = tokenizer.word_index

In [97]:
word2id

{'shall': 1,
 'unto': 2,
 'lord': 3,
 'thou': 4,
 'thy': 5,
 'god': 6,
 'ye': 7,
 'said': 8,
 'thee': 9,
 'upon': 10,
 'man': 11,
 'israel': 12,
 'king': 13,
 'son': 14,
 'hath': 15,
 'people': 16,
 'came': 17,
 'house': 18,
 'come': 19,
 'one': 20,
 'children': 21,
 'also': 22,
 'day': 23,
 'land': 24,
 'men': 25,
 'shalt': 26,
 'let': 27,
 'go': 28,
 'hand': 29,
 'saying': 30,
 'us': 31,
 'made': 32,
 'even': 33,
 'went': 34,
 'behold': 35,
 'saith': 36,
 'every': 37,
 'therefore': 38,
 'things': 39,
 'father': 40,
 'sons': 41,
 'hast': 42,
 'david': 43,
 'make': 44,
 'say': 45,
 'may': 46,
 'earth': 47,
 'jesus': 48,
 'great': 49,
 'name': 50,
 'thine': 51,
 'away': 52,
 'put': 53,
 'among': 54,
 'thereof': 55,
 'forth': 56,
 'give': 57,
 'neither': 58,
 'take': 59,
 'city': 60,
 'days': 61,
 'brought': 62,
 'moses': 63,
 'two': 64,
 'heart': 65,
 'pass': 66,
 'judah': 67,
 'jerusalem': 68,
 'according': 69,
 'know': 70,
 'took': 71,
 'thus': 72,
 'offering': 73,
 'bring': 74,
 'goo

In [50]:
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1
embed_size = 100

In [52]:
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_bible]

In [ ]:
#위 코드의 설명

for doc in norm_bible:
    for w in text.text_to_word_sequence(doc):
        print("토큰화된 단어:",w)
        print("토큰화된 단어의 인덱스:",word2id[w])
        print("===========")


In [55]:
print("Vocabulary Size:",vocab_size)
print("Vocabulary Sample:",list(word2id.items())[:10])

Vocabulary Size: 12425
Vocabulary Sample: [('shall', 1), ('unto', 2), ('lord', 3), ('thou', 4), ('thy', 5), ('god', 6), ('ye', 7), ('said', 8), ('thee', 9), ('upon', 10)]


In [82]:
from keras.preprocessing.sequence import skipgrams

skip_grams = [skipgrams(wid,vocabulary_size=vocab_size,window_size=10) for wid in wids]

In [83]:
pairs,labels = skip_grams[0][0],skip_grams[0][1]

In [96]:
wids[1]

[154, 2450, 13, 1154, 5766]

In [91]:
for i in range(10):
    print("{:s}({:d}),{:s}({:d})) -> {:d}".format(id2word[pairs[i][0]],pairs[i][0],id2word[pairs[i][1]],pairs[i][1],labels[i]))

james(1154),bible(5766)) -> 1
king(13),bible(5766)) -> 1
bible(5766),james(1154)) -> 1
james(1154),hawk(4763)) -> 0
king(13),james(1154)) -> 1
james(1154),king(13)) -> 1
king(13),messengers(731)) -> 0
james(1154),greediness(12102)) -> 0
bible(5766),stript(8595)) -> 0
bible(5766),eleph(9105)) -> 0
